In [219]:
# import necessary libraries
import numpy as np
import pandas as pd
import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from collections import defaultdict
from textwrap import wrap
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, mean_squared_error
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.preprocessing import StandardScaler
import nltk

# random
import warnings
warnings.filterwarnings('ignore')

In [220]:
# loading in 
df = pd.read_csv('full_df.csv')

In [221]:
# looking at df
print(df.shape)
df.head()

(6607, 18)


,Unnamed: 0,created_at,id,lng,lat,topic,sentiment,stance,gender,temperature_avg,aggressiveness,text,Full_Address,City,Continent,Country,State,is_english
0,0,2008-03-28 14:15:51+00:00,778602262,-93.101503,44.950404,Importance of Human Intervantion,-0.042726,believer,male,-4.082745,aggressive,"on march 29, 2008 at 8 pm, make a statement ab...","{'ISO_3166-1_alpha-2': 'US', 'ISO_3166-1_alpha...",Saint Paul,North America,United States,Minnesota,True
1,1,2008-03-28 15:36:30+00:00,778649364,-117.164720,32.715710,Global stance,0.429441,believer,undefined,-0.132076,not aggressive,City looks at green building standards: Hoping...,"{'ISO_3166-1_alpha-2': 'US', 'ISO_3166-1_alpha...",San Diego,North America,United States,California,True
2,2,2008-03-28 15:41:13+00:00,778652293,-122.355847,37.788497,Weather Extremes,0.092446,neutral,male,-2.324198,not aggressive,@thiskat @agray @payload Snow? In PDX? In Marc...,"{'ISO_3166-1_alpha-2': 'US', 'ISO_3166-1_alpha...",San Francisco,North America,United States,California,True
3,3,2008-03-28 15:56:42+00:00,778661751,-123.033121,44.939157,Ideological Positions on Global Warming,-0.337010,denier,male,-4.810226,not aggressive,someone alert al gore - global warming isn't w...,"{'ISO_3166-1_alpha-2': 'US', 'ISO_3166-1_alpha...",Salem,North America,United States,Oregon,False
4,4,2008-03-28 16:09:12+00:00,778669248,-123.364953,48.428318,Weather Extremes,-0.317469,denier,male,-4.862617,not aggressive,It's snowing in Langford and sticking! It's t...,"{'ISO_3166-1_alpha-2': 'CA', 'ISO_3166-1_alpha...",Victoria,North America,Canada,British Columbia,True


In [222]:
# looking at sentiments 
pos = (df['sentiment'] > 0.5).sum()
neg = (df['sentiment'] < -0.5).sum()
neutral = len(df) - pos - neg
print(f'Positive: {pos} Negative: {neg} Neutral: {neutral}')

Positive: 1014 Negative: 1021 Neutral: 4572


In [223]:
# logic to create classes for sentiment
sentiment  = df["sentiment"]

conditions = [sentiment >= 0.5,
              sentiment <= -0.5,
              (sentiment >-0.5) & (sentiment <0.5)]
choices  = [2,0,1]

# create a new column in the DF based on the conditions
df["label"] = np.select(conditions, choices, "ERROR")

In [224]:
# checking it out
df = df.drop('Unnamed: 0',axis=1)
df = df.drop('sentiment', axis=1)
df['label'] = df['label'].astype(int)

# dropping unwanted columns
df = df.drop('id', axis=1)
df = df.drop('lng', axis=1)
df = df.drop('Full_Address', axis=1)
df = df.drop('lat', axis=1)
df = df.drop('temperature_avg', axis=1)
df = df.drop('is_english', axis=1)
df = df.drop('created_at', axis=1)

df.head()

,topic,stance,gender,aggressiveness,text,City,Continent,Country,State,label
0,Importance of Human Intervantion,believer,male,aggressive,"on march 29, 2008 at 8 pm, make a statement ab...",Saint Paul,North America,United States,Minnesota,1
1,Global stance,believer,undefined,not aggressive,City looks at green building standards: Hoping...,San Diego,North America,United States,California,1
2,Weather Extremes,neutral,male,not aggressive,@thiskat @agray @payload Snow? In PDX? In Marc...,San Francisco,North America,United States,California,1
3,Ideological Positions on Global Warming,denier,male,not aggressive,someone alert al gore - global warming isn't w...,Salem,North America,United States,Oregon,1
4,Weather Extremes,denier,male,not aggressive,It's snowing in Langford and sticking! It's t...,Victoria,North America,Canada,British Columbia,1


In [225]:
# loooking at shape again
print(df.shape)

(6607, 10)


In [226]:
# splitting the data into independent and dependent variables
X = df.iloc[:,0:8].values
y = df.iloc[:,9].values
print('The independent features set: ')
print(X[:10,:])
print('The dependent variable: ')
print(y[:10])

The independent features set: 
[['Importance of Human Intervantion' 'believer' 'male' 'aggressive'
  'on march 29, 2008 at 8 pm, make a statement about climate change by turning off your lights for earth hour.'
  'Saint Paul' 'North America' 'United States']
 ['Global stance' 'believer' 'undefined' 'not aggressive'
  'City looks at green building standards: Hoping to do its part in reducing global warming, the city... http://tinyurl.com/37mphl'
  'San Diego' 'North America' 'United States']
 ['Weather Extremes' 'neutral' 'male' 'not aggressive'
  '@thiskat @agray @payload Snow? In PDX? In March?  Thanks global warming!'
  'San Francisco' 'North America' 'United States']
 ['Ideological Positions on Global Warming' 'denier' 'male'
  'not aggressive'
  "someone alert al gore - global warming isn't working!!" 'Salem'
  'North America' 'United States']
 ['Weather Extremes' 'denier' 'male' 'not aggressive'
  "It's snowing in Langford and sticking!  It's the end of March almost April.  I thou

In [227]:
# train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 21)
# X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.90, random_state=42)
# X_train, X_test, y_train, y_test = train_test_split(X_train,y_train,test_size = 0.5, random_state=42)
# X_val, X_test, y_val, y_test = train_test_split(X_test,y_test,test_size = 0.5, random_state=42)

In [228]:
df.head()

,topic,stance,gender,aggressiveness,text,City,Continent,Country,State,label
0,Importance of Human Intervantion,believer,male,aggressive,"on march 29, 2008 at 8 pm, make a statement ab...",Saint Paul,North America,United States,Minnesota,1
1,Global stance,believer,undefined,not aggressive,City looks at green building standards: Hoping...,San Diego,North America,United States,California,1
2,Weather Extremes,neutral,male,not aggressive,@thiskat @agray @payload Snow? In PDX? In Marc...,San Francisco,North America,United States,California,1
3,Ideological Positions on Global Warming,denier,male,not aggressive,someone alert al gore - global warming isn't w...,Salem,North America,United States,Oregon,1
4,Weather Extremes,denier,male,not aggressive,It's snowing in Langford and sticking! It's t...,Victoria,North America,Canada,British Columbia,1


In [229]:
# label encoding 
from sklearn import preprocessing

cols = ['topic', 'stance', 'gender', 'aggressiveness', 'City', 'Continent', 'Country', 'State']

le = preprocessing.LabelEncoder()

for col in cols:
    df[col] = le.fit_transform(df[col].astype(str))



In [230]:
df.head
a = df['Country'].unique()
print(a)

[61  6 60 27 42  1 13 30 37 20 58 57 29 44 28 26 34 54 38 25 52 19  3 18
 16 41 48  9 59 36 15 51 21 22  8  7 46 31 11 43 35 45 55 10 32  0  5 53
 40 33 24 17 12 23 49 47  2 50 56 14 39  4]


In [168]:
from sklearn.feature_extraction.text import TfidfVectorizer
v = TfidfVectorizer()
x = v.fit_transform(df['text'])

In [115]:
df.head

<bound method NDFrame.head of       topic  stance  gender  aggressiveness  \
0         1       1       1               1   
1         1       1       1               1   
2         1       1       1               1   
3         1       1       1               1   
4         1       1       1               1   
...     ...     ...     ...             ...   
6602      1       1       1               1   
6603      1       1       1               1   
6604      1       1       1               1   
6605      2       2       2               2   
6606      1       1       1               1   

                                                   text  City  Continent  \
0     on march 29, 2008 at 8 pm, make a statement ab...     1          1   
1     City looks at green building standards: Hoping...     1          1   
2     @thiskat @agray @payload Snow? In PDX? In Marc...     1          1   
3     someone alert al gore - global warming isn't w...     1          1   
4     It's snowing in Lan

In [110]:
import texthero as hero
df['tfidf'] = hero.tfidf(df['text'])

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')


In [91]:
# Feature Scaling
scaler = StandardScaler(with_mean=False)
X_train = scaler.fit_transform(X_train_tfidf)
X_test = scaler.transform(X_test_tfidf)

In [92]:
# Fitting Random Forest Classification to the Training set
classifier = RandomForestClassifier(n_estimators = 10, criterion = 'entropy', random_state = 42)
classifier.fit(X_train_tfidf, y_train)

ValueError: Found input variables with inconsistent numbers of samples: [42280, 5285]

In [76]:
# Predicting the Test set results
definitions = ['topic', 'stance', 'gender', 'aggressiveness', 'text', 'City', 'Continent', 'Country', 'State']
y_pred = classifier.predict(X_test)
reversefactor = dict(zip(range(3),definitions))
y_test = np.vectorize(reversefactor.get)(y_test)
y_pred = np.vectorize(reversefactor.get)(y_pred)

ValueError: Found input variables with inconsistent numbers of samples: [42280, 5285]